In [1]:
from bitarray import bitarray

def hammingDist(s1,s2):
    if len(s1) != len(s2):
        print("Inputs are different sizes")
    hdist = 0
    for i in range(len(s1)):
        if s1[i] != s2[i]:
            hdist = hdist + 1
    return hdist

In [2]:
from bitarray import bitarray

#Returns hamming dstance between templates and masks with no shift.
def updatedHammingDist(mask1,mask2,template1,template2):
    if len(template1) != len(template2):
        print("Inputs are different sizes")
        return -1
    if len(mask1) != len(template1):
        print("Mask 1 is the wrong size.")
        return -1
    if len(mask2) != len(template1):
        print("Mask 2 is the wrong size.")
        return -1
    mismatches = 0
    comparisons = 0
    for i in range(len(template1)):
        if mask1[i] == 0:
            if mask2[i] == 0:
                if template1[i] == template2[i]:
                    comparisons = comparisons + 1
                else:
                    comparisons = comparisons + 1
                    mismatches = mismatches + 1
    hdist = mismatches / comparisons
    #print(mask1,rightrot(mask1))
    return hdist

#Defines bit rotations on a bitarray. We don't need to encode right rotations because we can always rotate
#the other bitarray left and get the same effect. 
def leftrot(ba):
    u = ba[1:]
    if ba[0]:
        u.extend('1')
    else:
        u.extend('0')
    return u

def multiLeftrot(ba,n):
    u = ba
    for i in range(0,n):
        u = leftrot(u)
    return u
    

#Iterates through maxShifts shifts in mask 1 and template 1 in each direction and takes the minimum.
#We select 16 rotations in each direction, but this is totally arbitrary.
def bestDist(mask1,mask2,template1,template2,maxShifts):
    topHammingDist = 1
    for i in range(0,maxShifts + 1):
        
        #First rotate mask/template 1
        newM1 = multiLeftrot(mask1,i)
        newT1 = multiLeftrot(template1,i)
        currHammingDist = updatedHammingDist(newM1,mask2,newT1,template2)
        if currHammingDist < topHammingDist:
            topHammingDist = currHammingDist
        
        #... then rotate mask/template 2
        newM2 = multiLeftrot(mask2,i)
        newT2 = multiLeftrot(template2,i)
        currHammingDist = updatedHammingDist(mask1,newM2,template1,newT2)
        if currHammingDist < topHammingDist:
            topHammingDist = currHammingDist
            
    #if topHammingDist < .32:
        #print("Match!")
    return topHammingDist


In [5]:
import random
#Start with conf. matrix for correctly matching irises or not.

#Define cutoff.
cutoff = .44

print("Nonmatches")
#Irises should not match.
totalMatches = 0
for i in range(0,(108*3)):
    #Pick random non-matching irises.
    iA = random.randint(1, 108)
    iB = random.randint(1, 108)
    while iB == iA:
        iB = random.randint(1,108)
    kA = random.randint(1,2)
    kB = random.randint(1,2)
    jA = random.randint(1,3)
    jB = random.randint(1,3)
    uA = format(iA, '03d')
    uB = format(iB, '03d')
    
    #Open Files
    mfile1 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/maskexport_" + uA + "_" + str(kA) + "_" + str(jA) + ".txt", "r")
    mfile2 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/maskexport_" + uB + "_" + str(kB) + "_" + str(jB) + ".txt", "r")
    tfile1 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/templateexport_" + uA + "_" + str(kA) + "_" + str(jA) + ".txt", "r")
    tfile2 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/templateexport_" + uB + "_" + str(kB) + "_" + str(jB) + ".txt", "r")
    
    #Try to find best match.
    mf1 = bitarray(mfile1.read())
    mf2 = bitarray(mfile2.read())
    tf1 = bitarray(tfile1.read())
    tf2 = bitarray(tfile2.read())
    bd = bestDist(mf1,mf2,tf1,tf2,16)
    #print(bd)
    if bd < cutoff:
        totalMatches = totalMatches + 1
print("Match rate for nonmatches:",totalMatches/(108*3))

print("Matches:")
#Irises should match.
totalMatches = 0
for i in range(1,109):
    for j in range(1,4):
        u = format(i, '03d')
        mfile1 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/maskexport_" + u + "_1_" + str(j) + ".txt", "r")
        mfile2 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/maskexport_" + u + "_2_" + str(j) + ".txt", "r")
        tfile1 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/templateexport_" + u + "_1_" + str(j) + ".txt", "r")
        tfile2 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/templateexport_" + u + "_2_" + str(j) + ".txt", "r")
        mf1 = bitarray(mfile1.read())
        mf2 = bitarray(mfile2.read())
        tf1 = bitarray(tfile1.read())
        tf2 = bitarray(tfile2.read())
        bd = bestDist(mf1,mf2,tf1,tf2,16)
        if bd < cutoff:
            totalMatches = totalMatches + 1
        #print(bd)
print("Match rate for matches:",totalMatches/(108*3))


Nonmatches
Match rate for nonmatches: 0.033950617283950615
Matches:
Match rate for matches: 0.9907407407407407


In [3]:
import random
#Start with conf. matrix for correctly matching irises or not.
#This time, we do the procedure for small irises. 

#Define cutoff.
cutoff = .42

print("Nonmatches")
#Irises should not match.
totalMatches = 0
for i in range(0,(108*3)):
    #Pick random non-matching irises.
    iA = random.randint(1, 108)
    iB = random.randint(1, 108)
    while iB == iA:
        iB = random.randint(1,108)
    kA = random.randint(1,2)
    kB = random.randint(1,2)
    jA = random.randint(1,3)
    jB = random.randint(1,3)
    uA = format(iA, '03d')
    uB = format(iB, '03d')
    
    #Open Files
    mfile1 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/maskexport_" + uA + "_" + str(kA) + "_" + str(jA) + ".txt", "r")
    mfile2 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/resultsSmall/maskexport_" + uB + "_" + str(kB) + "_" + str(jB) + ".txt", "r")
    tfile1 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/templateexport_" + uA + "_" + str(kA) + "_" + str(jA) + ".txt", "r")
    tfile2 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/resultsSmall/templateexport_" + uB + "_" + str(kB) + "_" + str(jB) + ".txt", "r")
    
    #Try to find best match.
    mf1 = bitarray(mfile1.read())
    mf2 = bitarray(mfile2.read())
    tf1 = bitarray(tfile1.read())
    tf2 = bitarray(tfile2.read())
    bd = bestDist(mf1,mf2,tf1,tf2,16)
    #print(bd)
    if bd < cutoff:
        totalMatches = totalMatches + 1
print("Match rate for small nonmatches:",totalMatches/(108*3))

print("Matches:")
#Irises should match.
totalMatches = 0
for i in range(1,109):
    for j in range(1,4):
        u = format(i, '03d')
        mfile1 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/maskexport_" + u + "_1_" + str(j) + ".txt", "r")
        mfile2 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/resultsSmall/maskexport_" + u + "_2_" + str(j) + ".txt", "r")
        tfile1 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/templateexport_" + u + "_1_" + str(j) + ".txt", "r")
        tfile2 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/resultsSmall/templateexport_" + u + "_2_" + str(j) + ".txt", "r")
        mf1 = bitarray(mfile1.read())
        mf2 = bitarray(mfile2.read())
        tf1 = bitarray(tfile1.read())
        tf2 = bitarray(tfile2.read())
        bd = bestDist(mf1,mf2,tf1,tf2,16)
        if bd < cutoff:
            totalMatches = totalMatches + 1
        #print(bd)
print("Match rate for small matches:",totalMatches/(108*3))
print("Error for matches:",1-(totalMatches/(108*3)))

Nonmatches
Match rate for small nonmatches: 0.006172839506172839
Matches:
Match rate for small matches: 0.9660493827160493
Error for matches: 0.03395061728395066


In [4]:
#Test ability to detect if iris i is in a database of 1/2 of all irises.
import random
from bitarray import bitarray

cutoff = .40

#Create integers to pull from
integers = []
for i in range(108):
    integers.append(i+1)
    
inPopAttempts = 0
inPopCorrect = 0
outPopAttempts = 0
outPopCorrect = 0

for i in range(324):
    population = random.sample(integers, 54)
    iris = random.choice(integers)
    
    #Determine if we think iris is in population or not.
    uIris = format(iris, '03d')
    mfileIris = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/maskexport_" + uIris + "_2_1" + ".txt", "r")
    tfileIris = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/templateexport_" + uIris + "_2_1" + ".txt", "r")
    mf2 = bitarray(mfileIris.read())
    tf2 = bitarray(tfileIris.read())
    isInPopGuess = 0
    minHammingDist = 1
    for j in population:
        uA = format(j, '03d')
        mfile1 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/maskexport_" + uA + "_1_1" + ".txt", "r")
        tfile1 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/templateexport_" + uA + "_1_1" + ".txt", "r")
        mf1 = bitarray(mfile1.read())
        tf1 = bitarray(tfile1.read())
        bd = bestDist(mf1,mf2,tf1,tf2,8)
        if bd < cutoff:
            if bd < minHammingDist:
                minHammingDist = bd
                isInPopGuess = j
    
    #See if we're right
    if iris in population:
        inPopAttempts = inPopAttempts + 1
        if isInPopGuess == iris:
            inPopCorrect = inPopCorrect + 1
    else:
        outPopAttempts = outPopAttempts + 1
        if isInPopGuess == 0:
            outPopCorrect = outPopCorrect + 1
    print(i)     
#Print Results
print(inPopAttempts,inPopCorrect,outPopAttempts,outPopCorrect)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
#Test ability to detect if iris i is in a database of 1/2 of all irises.
import random
from bitarray import bitarray

cutoffs = [.38,.39,.4,.41,.42,.43]

#Create integers to pull from

for cutoff in cutoffs:
    inPopAttempts = 0
    inPopCorrect = 0
    outPopAttempts = 0
    outPopCorrect = 0
    
    integers = []
    for i in range(108):
        integers.append(i+1)
    

    for i in range(324):
        population = random.sample(integers, 54)
        iris = random.choice(integers)

        #Determine if we think iris is in population or not.
        uIris = format(iris, '03d')
        mfileIris = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/maskexport_" + uIris + "_2_1" + ".txt", "r")
        tfileIris = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/templateexport_" + uIris + "_2_1" + ".txt", "r")
        mf2 = bitarray(mfileIris.read())
        tf2 = bitarray(tfileIris.read())
        isInPopGuess = 0
        minHammingDist = 1
        for j in population:
            uA = format(j, '03d')
            mfile1 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/maskexport_" + uA + "_1_1" + ".txt", "r")
            tfile1 = open("/Users/Tory/Documents/Documents - MacBook Pro (4)/results/templateexport_" + uA + "_1_1" + ".txt", "r")
            mf1 = bitarray(mfile1.read())
            tf1 = bitarray(tfile1.read())
            bd = bestDist(mf1,mf2,tf1,tf2,8)
            if bd < cutoff:
                if bd < minHammingDist:
                    minHammingDist = bd
                    isInPopGuess = j

        #See if we're right
        if iris in population:
            inPopAttempts = inPopAttempts + 1
            if isInPopGuess == iris:
                inPopCorrect = inPopCorrect + 1
        else:
            outPopAttempts = outPopAttempts + 1
            if isInPopGuess == 0:
                outPopCorrect = outPopCorrect + 1
        #print(i)     
    #Print Results
    print(cutoff,inPopCorrect/inPopAttempts,outPopCorrect/outPopAttempts)

0.38 0.9476744186046512 1.0
0.39 0.8866666666666667 1.0
0.4 0.9263803680981595 1.0
0.41 0.9807692307692307 0.9880952380952381
0.42 0.9790209790209791 0.9281767955801105


KeyboardInterrupt: 